In [48]:
from langchain_core.messages import HumanMessage, BaseMessage
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langgraph.graph import END, START, StateGraph, add_messages
from pydantic import BaseModel, Field
from typing import List

class MadrsFieldRating(BaseModel):
    score: int = Field(
        description="The numerical score for this field.",
        ge=0,
        le=6,
    )
    justification: str = Field(
        description="A brief justification for the score given for this field."
    )

class MadrsRating(BaseModel):
    apparent_sadness: MadrsFieldRating = Field(
        description = "The MADRS rating for apparent sadness."
    )
    reported_sadness: MadrsFieldRating = Field(
        description = "The MADRS rating for reported sadness."
    )
    inner_tension: MadrsFieldRating = Field(
        description = "The MADRS rating for inner tension."
    )
    reduced_sleep: MadrsFieldRating = Field(
        description = "The MADRS rating for reduced sleep."
    )
    reduced_appetite: MadrsFieldRating = Field(
        description = "The MADRS rating for reduced appetite."
    )
    concentration_difficulties: MadrsFieldRating = Field(
        description = "The MADRS rating for concentration difficulties."
    )
    lassitude: MadrsFieldRating = Field(
        description = "The MADRS rating for lassitude."
    )
    inability_to_feel: MadrsFieldRating = Field(
        description = "The MADRS rating for inability to feel."
    )
    pessimistic_thoughts: MadrsFieldRating = Field(
        description = "The MADRS rating for pessimistic thoughts."
    )
    suicidal_thoughts: MadrsFieldRating = Field(
        description = "The MADRS rating for suicidal thoughts."
    )

In [57]:
llm = ChatOpenAI(
    model = "deepseek-r1-distill-llama-70B",
    base_url = "http://localhost:8080/v1",
    api_key = "NULL",
)

In [58]:
messages : List[BaseMessage] = list()
with open("../data/interviews/transcripts/corrected/MAD-001.txt") as transcript_file:
    transcript = transcript_file.read()
with open("../data/madrs-instructions/primer.md") as primer_file:
    primer = primer_file.read()
prompt_template = PromptTemplate.from_template(
    """
    You are an expert in psychiatric assessments and depression. For your
    recollection, consider the following primer on MADRS interviews:

    ---

    {primer}

    ---

    Using this information, provide the most accurate assessment of the
    following transcription of a MADRS interview. Provide your scores for
    each of the items described in the primer, providing a best guess for
    apparent sadness, together with a brief justification/discussion.

    ---

    {transcript}
    """
)

In [59]:
messages += prompt_template.invoke(
    {
        "transcript": transcript,
        "primer": primer,
    }
).to_messages()
response = llm.invoke(messages)

In [72]:
MadrsRating.model_validate_json(parse_response.content)

MadrsRating(apparent_sadness=MadrsFieldRating(score=3, justification="The patient describes their mood as a 'low mood' that can brighten with effort, indicating a moderate level of sadness."), reported_sadness=MadrsFieldRating(score=3, justification='The patient reports feeling sad much of the time but can still cheer up with effort.'), inner_tension=MadrsFieldRating(score=4, justification='The patient experiences situational anxiety and panic, particularly during stressful workdays.'), reduced_sleep=MadrsFieldRating(score=3, justification='The patient has difficulty falling asleep and staying asleep, waking up multiple times a night.'), reduced_appetite=MadrsFieldRating(score=4, justification='The patient has a reduced appetite during the day, eating minimally, and food does not taste good.'), concentration_difficulties=MadrsFieldRating(score=3, justification='The patient has some difficulty concentrating, especially after work due to stress and multitasking.'), lassitude=MadrsFieldRa

In [68]:
structured_llm = llm.bind(extra_body={
    "response_format": {
        "type": "json_schema",
        "json_schema": {"name": "MadrsAssessment", "schema": MadrsRating.model_json_schema(), "strict": True}
    }
})
parse_message = HumanMessage("Please format your previous response in the prescibed way.")
parse_response = structured_llm.invoke(messages + [response, parse_message])


TypeError: 'AIMessage' object is not subscriptable